## Importing dependencies

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


## Initialize workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ED4FP5FXN to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-141597
aml-quickstarts-141597
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


## Auto ML Experiment

In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'data_quality_analysis'
project_folder = './catalogue-scorecard-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
data_quality_analysis,quick-starts-ws-141597,Link to Azure Machine Learning studio,Link to Documentation


## Create an AmlCompute cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "comp"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_v2',#for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Data import

In [5]:
import pandas as pd

auto_ds_path="https://raw.githubusercontent.com/shatakshipachori/data_quality_analysis/main/catalogue-scorecard.csv"
ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)


## AutoML Configuration

In [6]:
import logging
import time

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":2,
    "primary_metric":'normalized_root_mean_squared_error',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":15,
    "max_concurrent_iterations":5,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='regression',
    compute_target='comp',
    training_data=ds,
    label_column_name='score',
    **automl_settings
    )

## Submit the experiment

In [8]:
automl_run=experiment.submit(config=automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on comp with default configuration
Running on remote compute: comp
Parent Run ID: AutoML_aa54d0b1-9cc7-4363-b8af-d7ce9e6d9c5f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality f

## Run Details

In [9]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary de

{'runId': 'AutoML_aa54d0b1-9cc7-4363-b8af-d7ce9e6d9c5f',
 'target': 'comp',
 'status': 'Completed',
 'startTimeUtc': '2021-03-29T19:04:46.94355Z',
 'endTimeUtc': '2021-03-29T19:19:55.943365Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'comp',
  'AMLSettingsJsonString': '{"path":null,"name":"data_quality_analysis","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-141597","workspace_name":"quick-starts-ws-141597","region":"southcentralus","compute_target":"comp","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"normalized_root_mean_squared_error","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_val

## Best Model

In [10]:
# Retrieve and save your best automl model.
best_automl_run,best_automl_model=automl_run.get_output()
best_run_metrics=automl_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection', 'BestRunExplainModel', 'ModelExplanationDataSetSetup', 'PickSurrogateModel', 'EngineeredFeatureExplanations', 'EngineeredFeatureExplanations', 'RawFeaturesExplanations', 'FailedModelExplanations']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.', 'Best run model explanations started', 'Model explanations data setup completed', 'Choosing LightGBM as the surrogate model for explanations', 'Computation of engineered features started', 'Computation of engineered features completed', 'Computation of raw features started', 'Failed to explain best model']
explained_variance 0.999940

In [11]:
print(best_automl_run)
best_automl_run

Run(Experiment: data_quality_analysis,
Id: AutoML_aa54d0b1-9cc7-4363-b8af-d7ce9e6d9c5f_0,
Type: azureml.scriptrun,
Status: Completed)


Experiment,Id,Type,Status,Details Page,Docs Page
data_quality_analysis,AutoML_aa54d0b1-9cc7-4363-b8af-d7ce9e6d9c5f_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Classification using kNN

In [58]:
from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder
# define data
ds = asarray([['Gold'], ['Silver'], ['Bronze']])
print(ds)
# define ordinal encoding
encoder = OrdinalEncoder()
# transform data
result = encoder.fit_transform(ds)
print(result)

[['Gold']
 ['Silver']
 ['Bronze']]
[[1.]
 [2.]
 [0.]]


In [65]:
ds=ds.drop_columns('package','grade')
def data(df):
    y=df.pop('grade_norm')
    X=df
    return X,y

ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)
ds= ds.to_pandas_dataframe()
X,y=data(ds)


AttributeError: 'DataFrame' object has no attribute 'drop_columns'

## Import the class

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import argparse
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

## Instantiate the estimator

In [35]:
knn=KNeighborsClassifier(n_neighbors=5)
print(knn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


## Model Training

In [44]:
type(X)

pandas.core.frame.DataFrame

In [40]:
knn.fit(X,y)
except ValueError,e:
        print "error",e,"on line",i

SyntaxError: invalid syntax (<ipython-input-40-e94cfc61570c>, line 2)

## Model Prediction